# MaiChart AI - Cloud Training (Colab & Kaggle)

1. **Check GPU**: 
    - **Colab**: Runtime -> Change runtime type -> T4 GPU.
    - **Kaggle**: Settings -> Accelerator -> T4 GPU (or P100).
2. **Features**:
    *   **Auto-Resume**: 自动寻找最新的 `epoch_X.pt` 并接力轮数训练。
    *   **Environment Aware**: 自动检测 Colab 或 Kaggle 环境并配置路径。
    *   **Cloud Persistence**: Colab 自动挂载 Google Drive，Kaggle 默认保存到 Working Directory。
    *   **Deep Integration**: 自动处理数据集解压、依赖安装及断点续传。

In [1]:
!nvidia-smi

Wed Jan 21 18:03:59 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import shutil
import glob

# 环境检测 / Environment Detection
is_colab = 'google.colab' in str(get_ipython())
is_kaggle = os.path.exists('/kaggle/working')

if is_colab:
    ROOT_PATH = '/content'
    DEFAULT_ZIP_PATH = '/content/drive/MyDrive/maimai_data.zip'
    DEFAULT_STORAGE_PATH = '/content/drive/MyDrive/maimai/models'
elif is_kaggle:
    ROOT_PATH = '/kaggle/working'
    DEFAULT_ZIP_PATH = '/kaggle/input/maimai-data/maimai_data.zip'
    DEFAULT_STORAGE_PATH = '/kaggle/working/models'
else:
    ROOT_PATH = os.getcwd()
    DEFAULT_ZIP_PATH = './maimai_data.zip'
    DEFAULT_STORAGE_PATH = './models'

os.environ['ROOT_PATH'] = ROOT_PATH
os.environ['PROJECT_PATH'] = os.path.join(ROOT_PATH, 'maimai-chat-ai')

print(f"Detected Environment: {'Colab' if is_colab else 'Kaggle' if is_kaggle else 'Local'}")

# 1. 确保回到根目录
os.chdir(ROOT_PATH)
if os.path.exists('maimai-chat-ai'):
    print("Removing existing project folder...")
    shutil.rmtree('maimai-chat-ai')

# 2. 重新克隆仓库
print("Cloning repository...")
!git clone https://github.com/lvyirgil/maimai-chat-ai.git

# 验证克隆是否成功
if not os.path.exists('maimai-chat-ai'):
    print("\n❌ Error: Failed to clone repository.")
    if is_kaggle:
        print("💡 Hint: Please ensure 'Internet' is set to 'On' in the Kaggle sidebar settings.")
    raise FileNotFoundError("Project directory not found after git clone.")

# 3. 进入文件夹
os.chdir(os.environ['PROJECT_PATH'])

# 4. 检查文件列表
print("\n--- Project Files ---")
!ls

# 5. 安装依赖
if os.path.exists('requirements.txt'):
    !pip install -r requirements.txt
else:
    print("\n❌ Error: requirements.txt not found.")

/content
Cloning into 'maimai-chat-ai'...
remote: Enumerating objects: 1883, done.
remote: Counting objects: 100% (1883/1883), done.
remote: Compressing objects: 100% (1840/1840), done.
remote: Total 1883 (delta 42), reused 1874 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (1883/1883), 4.39 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/maimai-chat-ai

--- 当前目录下的文件 ---
check_cuda.py  GPU_TRAINING.md		 README.md	   tests
configs        IMPORT_TOOL_INSTALLED.md  requirements.txt  train_gpu.bat
data	       models			 scripts	   train_gpu.py
docs	       notebooks		 src		   train.ps1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 86.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
import zipfile
import os
import shutil
import glob

# 尝试自动定位 zip 文件
zip_path = DEFAULT_ZIP_PATH
processed_zip_path = '/content/drive/MyDrive/processed_data.zip' if is_colab else './processed_data.zip'
skip_preprocess = False

# 1. 优先检查是否存在预处理过的数据包 (processed_data.zip)
if os.path.exists(processed_zip_path):
    print(f"📦 发现已处理的数据包: {processed_zip_path}")
    processed_dir = os.path.join(os.environ['PROJECT_PATH'], 'data', 'processed')
    os.makedirs(processed_dir, exist_ok=True)
    
    print("正在解压预处理后的数据到 data/processed...")
    # 使用 python zipfile 确保跨平台兼容性（虽然 Colab/Kaggle 都是 Linux）
    with zipfile.ZipFile(processed_zip_path, 'r') as z:
        z.extractall(processed_dir)
        
    print(f"✅ 预处理数据已就绪，当前样本数: {len(os.listdir(processed_dir))}")
    skip_preprocess = True

# 2. 如果没有预处理过的数据，则解压原始数据并准备进行预处理
if not skip_preprocess:
    if not os.path.exists(zip_path):
        print(f"⚠️ 默认原始数据路径 {zip_path} 未找到，尝试搜索...")
        if is_kaggle:
            search_paths = glob.glob('/kaggle/input/**/*.zip', recursive=True)
            if search_paths:
                zip_path = search_paths[0]
                print(f"🔍 找到原始 Zip: {zip_path}")
        elif is_colab:
            print("❌ 未找到 maimai_data.zip 或 processed_data.zip。")

    target_base = os.path.join(os.environ['PROJECT_PATH'], 'data')

    def force_unzip(path):
        if not os.path.exists(path):
            print(f"❌ 错误：未找到 zip 文件: {path}")
            return

        # 清理文件夹
        for d in ['audio', 'raw']:
            shutil.rmtree(os.path.join(target_base, d), ignore_errors=True)
            os.makedirs(os.path.join(target_base, d), exist_ok=True)

        print(f"正在从 {path} 解压原始数据...")
        with zipfile.ZipFile(path, 'r') as z:
            for info in z.infolist():
                try:
                    name = info.filename.encode('cp437').decode('gbk')
                except:
                    name = info.filename
                name = name.replace('\\', '/')

                if name.lower().endswith(('.mp3', '.wav', '.ogg', '.flac')):
                    fname = os.path.basename(name)
                    if fname:
                        with z.open(info) as source, open(os.path.join(target_base, 'audio', fname), 'wb') as target:
                            shutil.copyfileobj(source, target)
                elif name.lower().endswith('.txt'):
                    fname = os.path.basename(name)
                    if fname:
                        with z.open(info) as source, open(os.path.join(target_base, 'raw', fname), 'wb') as target:
                            shutil.copyfileobj(source, target)

        print(f"✅ 原始数据提取完成！")
        print(f"音频数: {len(os.listdir(os.path.join(target_base, 'audio')))}, 谱面数: {len(os.listdir(os.path.join(target_base, 'raw')))}")

    force_unzip(zip_path)

# 将 skip_preprocess 导出到环境变量供下个单元格使用
os.environ['SKIP_PREPROCESS'] = 'True' if skip_preprocess else 'False'

正在强力解压中，请稍后（4.6GB 可能需要 2-3 分钟）...
✅ 解压完成！
当前音频数: 1757
当前谱面数: 1810


In [ ]:
os.chdir(os.environ['PROJECT_PATH'])

# 如果已经解压了 processed_data.zip，则跳过该步骤
if os.environ.get('SKIP_PREPROCESS') == 'True':
    print("⏭️ 检测到已处理数据，跳过预处理步骤。")
else:
    print("🚀 开始数据预处理...")
    !python -m src.data.preprocess

警告: 找不到 M@GICAL☆CURE! LOVE ♥ SHOT! 的音频文件
警告: 找不到 Freak Out Hr 的音频文件
警告: 找不到 デビルじゃないもん 的音频文件
警告: 找不到 てらてら 的音频文件
警告: 找不到 Ref_rain (for 7th Heaven) 的音频文件
警告: 找不到 Colorfull_Encounter 的音频文件
警告: 找不到 美夜月鏡 的音频文件
警告: 找不到 MYTH Re：LEASE 的音频文件
警告: 找不到 はいよろこんで 的音频文件
警告: 找不到 sølips 的音频文件
警告: 找不到 廃墟にいますキャンペーン 的音频文件
警告: 找不到 Flashback 的音频文件
警告: 找不到 Hainuwele 的音频文件
警告: 找不到 抜錨 的音频文件
警告: 找不到 お呪い 的音频文件
警告: 找不到 [嘘] ライアーダンサー 的音频文件
警告: 找不到 Chronomia 的音频文件
警告: 找不到 あいたい星人 的音频文件
警告: 找不到 HYP3RTRIBE 的音频文件
警告: 找不到 ㋰責任集合体 的音频文件
警告: 找不到 カーニバルハッピー 的音频文件
警告: 找不到 唱 的音频文件
警告: 找不到 アンダーキッズ 的音频文件
警告: 找不到 ハオ 的音频文件
警告: 找不到 しゅ～しん？変身☆ハカイシンzzZ 的音频文件
警告: 找不到 Pixel Galaxy 的音频文件
警告: 找不到 偉大なる悪魔は実は大天使パトラちゃん様なのだ！ 的音频文件
警告: 找不到 マツケンサンバⅡ 的音频文件
警告: 找不到 Beginning together! 的音频文件
警告: 找不到 ビビデバ 的音频文件
警告: 找不到 [X] 人マニア 的音频文件
警告: 找不到 Lover_s Trick 的音频文件
警告: 找不到 つづみぐさ 的音频文件
警告: 找不到 サウンドプレイヤー 的音频文件
警告: 找不到 シスターシスター 的音频文件
警告: 找不到 FLΛME-FRΦST 的音频文件
警告: 找不到 Divide et impera! 的音频文件
警告: 找不到 シュガーホリック 的音频文件
警告: 找不到 NO ONE YES MAN 的音频文件
警告: 找不到 Bri

In [ ]:
import os
import glob
import shutil

# 1. 定义存储路径
drive_models_path = DEFAULT_STORAGE_PATH
local_models_path = os.path.join(os.environ['PROJECT_PATH'], 'models')

# 2. 只有在 Colab 时挂载网盘
if is_colab:
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
    
    # 建立软链接，确保训练器能直接保存到网盘
    os.makedirs(drive_models_path, exist_ok=True)
    if os.path.exists(local_models_path) and not os.path.islink(local_models_path):
        print("Moving local models to Drive sync folder...")
        # 使用 python 命令避免 shell 差异
        for f in glob.glob(f"{local_models_path}/*"):
            dest = os.path.join(drive_models_path, os.path.basename(f))
            if not os.path.exists(dest):
                shutil.move(f, dest)
        shutil.rmtree(local_models_path)
    
    if not os.path.islink(local_models_path):
        os.symlink(drive_models_path, local_models_path)
    print(f"✅ Colab 同步已就绪: {drive_models_path}")

elif is_kaggle:
    os.makedirs(local_models_path, exist_ok=True)
    print(f"✅ Kaggle 模式: 模型将保存在 {local_models_path}")
else:
    os.makedirs(local_models_path, exist_ok=True)

# 3. 自动寻找最新的 epoch 检查点进行恢复
checkpoints = glob.glob(f"{local_models_path}/epoch_*.pt")
latest_resume = ""
if checkpoints:
    try:
        # 按照数字大小排序找到最后一个 epoch
        latest_resume = max(checkpoints, key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))
        print(f"检测到最新检查点: {latest_resume}，将从该轮数继续训练。")
    except:
        pass
        
if not latest_resume and os.path.exists(f"{local_models_path}/best.pt"):
    latest_resume = f"{local_models_path}/best.pt"
    print("未找到 epoch 检查点，但检测到 best.pt，将从最佳状态恢复。")

# 4. 启动训练
os.chdir(os.environ['PROJECT_PATH'])
resume_cmd = f"--resume {latest_resume}" if latest_resume else ""
!python train_gpu.py --config configs/default.yaml {resume_cmd} --epochs 100

正在建立实时同步软链接...
✅ 实时同步已就绪：现在模型保存会直接写入网盘。
载入配置文件: configs/default.yaml

GPU 信息
✓ CUDA 可用
✓ GPU 数量: 1

GPU 0: Tesla T4
  - 计算能力: 7.5
  - 总显存: 14.7 GB
  - 当前显存占用: 0.0 GB

✓ PyTorch 版本: 2.9.0+cu126
✓ 混合精度: 启用 (FP16)

训练配置:
  - 设备: cuda
  - Batch Size: 4
  - 学习率: 1e-4
  - 混合精度: 启用
  - 最大轮数: 100
  - 梯度累积步数: 4

✓ 检测到 GPU，启用混合精度和优化参数
加载样本: 100% 1757/1757 [00:52<00:00, 33.59it/s]
加载了 1757 个样本
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
从 /content/maimai-chat-ai/models/best.pt 加载检查点 (epoch 5)
开始训练，设备: cuda
训练集大小: 1405
验证集大小: 175
Epoch 0:   0% 0/351 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 0: 352it [09:17,  1.58s/it, loss=0.2331, lr=1.00e-04]
Epoch 0: 

In [ ]:
os.chdir(os.environ['PROJECT_PATH'])
!git fetch origin && git reset --hard origin/main

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 401 bytes | 401.00 KiB/s, done.
From https://github.com/lvyirgil/maimai-chat-ai
   af80070..12043b4  main       -> origin/main
HEAD is now at 12043b4 Add --resume argument support to train_gpu.py
